# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import os
import cassandra

#### Creating list of filepaths to process original event csv data files

In [34]:
# Write your code here
all_csv_paths = list()
for sb, dr, files in os.walk("youtube_event_data/"):
    for file in files:
        if file.endswith(".csv"):
            all_csv_paths.append(os.path.join(sb,file))
            
print(len(all_csv_paths))
all_csv_paths[0]

30


'youtube_event_data/2018-11-01-events.csv'

#### Processing the CSV files to create the new `youtube_events_data_new.csv` file

In [76]:
# Write your code here
youtube_events_data_new = pd.DataFrame()

for file in all_csv_paths[0:]:
    df = pd.read_csv(file)
    youtube_events_data_new = youtube_events_data_new.append(df)

youtube_events_data_new.head(10)

,youtuber,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,video,status,ts,userId
0,NaN,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540920e+12,38,NaN,200,1.541110e+12,39.0
1,NaN,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540340e+12,139,NaN,200,1.541110e+12,8.0
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,You Gotta Be,200,1.541110e+12,8.0
3,NaN,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540340e+12,139,NaN,200,1.541110e+12,8.0
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Flat 55,200,1.541110e+12,8.0
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Quem Quiser Encontrar O Amor,200,1.541110e+12,8.0
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Eriatarka,200,1.541110e+12,8.0
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Becoming Insane,200,1.541110e+12,8.0
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Congratulations,200,1.541110e+12,8.0
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextVideo,1.540340e+12,139,Once again,200,1.541110e+12,8.0


In [79]:
# check the number of rows in your new csv file
youtube_events_data_new.to_csv("youtube_events_data_new.csv")
len(youtube_events_data_new)

8056

# Part II. Apache Cassandra coding portion of the project. 

## The new CSV file titled `youtube_events_data_new.csv`, The event_datafile_new.csv contains the following columns: 

- youtuber 
- video
- length
- sessionId
- ItemInSession
- firstName 
- lastName
- userId


.
.
.


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**youtube_events_data_new.csv**</font> after the code above is run:<br>

<img src="images/1.PNG"> 

In [81]:
cols = ['youtuber', 'video', 'length', 'sessionId', 'itemInSession', 'firstName', 'lastName', 'userId']
event_data = youtube_events_data_new[cols]
event_data.head()

,youtuber,video,length,sessionId,itemInSession,firstName,lastName,userId
0,NaN,NaN,NaN,38,0,Walter,Frye,39.0
1,NaN,NaN,NaN,139,0,Kaylee,Summers,8.0
2,Des'ree,You Gotta Be,246.30812,139,1,Kaylee,Summers,8.0
3,NaN,NaN,NaN,139,2,Kaylee,Summers,8.0
4,Mr Oizo,Flat 55,144.03873,139,3,Kaylee,Summers,8.0


In [83]:
event_data.dropna(axis=0, inplace=True)
event_data.head()

C:\Users\Nauman\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,youtuber,video,length,sessionId,itemInSession,firstName,lastName,userId
2,Des'ree,You Gotta Be,246.30812,139,1,Kaylee,Summers,8.0
4,Mr Oizo,Flat 55,144.03873,139,3,Kaylee,Summers,8.0
5,Tamba Trio,Quem Quiser Encontrar O Amor,177.18812,139,4,Kaylee,Summers,8.0
6,The Mars Volta,Eriatarka,380.42077,139,5,Kaylee,Summers,8.0
7,Infected Mushroom,Becoming Insane,440.26730,139,6,Kaylee,Summers,8.0


In [86]:
print(event_data.shape)
event_data.to_csv("event_data_file.csv")

(6820, 8)


In [3]:
event_data = pd.read_csv("event_data_file.csv", index_col=0)
event_data.head()

,youtuber,video,length,sessionId,itemInSession,firstName,lastName,userId
2,Des'ree,You Gotta Be,246.30812,139,1,Kaylee,Summers,8.0
4,Mr Oizo,Flat 55,144.03873,139,3,Kaylee,Summers,8.0
5,Tamba Trio,Quem Quiser Encontrar O Amor,177.18812,139,4,Kaylee,Summers,8.0
6,The Mars Volta,Eriatarka,380.42077,139,5,Kaylee,Summers,8.0
7,Infected Mushroom,Becoming Insane,440.26730,139,6,Kaylee,Summers,8.0


#### Creating a Cluster

In [5]:
# Create a Cassandra Cluster
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)
    

#### Create Keyspace `youtubedb`

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS youtubedb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace `youtubedb`

In [7]:
try:
    session.set_keyspace('youtubedb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Select the youtuber, video title and video's length in the YouTube app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Select only the following: name of youtuber, video (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Select every user name (first and last) in my YouTube app history who watched the video 'All Hands Against His Own'

In [11]:
##Q1 Making table
q1 = "CREATE TABLE IF NOT EXISTS table_test \
    ( youtuber text, video_title text, video_length float, sessionId int, itemInSession int, \
    PRIMARY KEY (sessionId, itemInSession) ) ;"
session.execute(q1)

**QUERY 1**
1. Select the youtuber, video title and video's length in the YouTube app history that was heard during sessionId = 338, and itemInSession = 4

In [12]:
##Q1 inserting data
qi1 = "INSERT INTO table_test (youtuber, video_title, video_length, sessionId, itemInSession) VALUES (%s, %s, %s, %s, %s)" 
for ind, row in event_data.iterrows():
    try:
        session.execute(qi1,(row['youtuber'], row['video'], float(row['length']), int(row['sessionId']), int(row['itemInSession']) ))
    except Exception as e:
        print(e)
        

In [13]:
##Q1 querying for results
res_q1 = "Select youtuber, video_title, video_length FROM table_test WHERE sessionId=338 AND itemInSession=4;"
rows = session.execute(res_q1)

for row in rows:
    print(row)

Row(youtuber='Faithless', video_title='Music Matters (Mark Knight Dub)', video_length=495.30731201171875)


**QUERY 2**
2. Select only the following: name of youtuber, video (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [116]:
##Q2 Making table
q2 = "CREATE TABLE IF NOT EXISTS table2 \
    ( youtuber text, video_title text, firstName text, lastName text, userId int, sessionId int, itemInSession int, \
    PRIMARY KEY ((userId, sessionId), itemInSession )) ;"
session.execute(q2)

In [117]:
##Q2 inserting data
qi2 = "INSERT INTO table2 (youtuber, video_title, firstName, lastName, userId, sessionId, itemInSession) VALUES (%s, %s, %s, %s, %s, %s, %s)" 
for ind, row in event_data.iterrows():
    try:
        session.execute(qi2,(row['youtuber'], row['video'], row['firstName'], row['lastName'], int(row['userId']), int(row['sessionId']), int(row['itemInSession']) ))
    except Exception as e:
        print(e)
        

In [118]:
##Q2 querying for results
res_q2 = '''Select youtuber, video_title, firstName ,lastName, itemInSession FROM table2
    WHERE userId=10 AND sessionId=182 ORDER BY itemInSession ; '''
rows = session.execute(res_q2)

for row in rows:
    print(row)

Row(youtuber='Down To The Bone', video_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)
Row(youtuber='Three Drives', video_title='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(youtuber='Sebastien Tellier', video_title='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(youtuber='Lonnie Gordon', video_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)


**QUERY 3**
3. Select every user name (first and last) in my YouTube app history who watched the video 'All Hands Against His Own'

In [131]:
##Q3 Making table
q3 = "CREATE TABLE IF NOT EXISTS table3 ( firstName text, lastName text, video_title text, \
    PRIMARY KEY (video_title, firstName, lastName ) ) ;"

session.execute(q3)

In [132]:
##Q3 inserting data
qi3 = "INSERT INTO table3 (firstName, lastName,video_title) VALUES (%s, %s, %s)" 
for ind, row in event_data.iterrows():
    try:
        session.execute(qi3,(row['firstName'], row['lastName'], row['video'] ))
    except Exception as e:
        print(e)
        

In [133]:
##Q3 querying for results
res_q3 = "Select firstName, lastName FROM table3 WHERE video_title = 'All Hands Against His Own' ;"
rows = session.execute(res_q3)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [136]:
q1 = "drop table table1"
q2 = "drop table table2"
q3 = "drop table table3"

session.execute(q1)

session.execute(q2)

session.execute(q3)

### Close the session and cluster connection¶

In [137]:
session.shutdown()

In [138]:
cluster.shutdown()